# Advanced RAG

In [34]:
question = "I would like to get esim. What should I do?"

In [46]:
# Import necessary libraries
import os
import numpy as np
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, FewShotChatMessagePromptTemplate
from langchain.load import dumps, loads
from operator import itemgetter

# Load API Keys from environment variables
load_dotenv()  # Load environment variables from a .env file

# Retrieve API keys from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")  # This key is loaded but not used in the code

# Initialize the chat model and embedding model
# ChatOpenAI is used to interact with the OpenAI GPT model, and OpenAIEmbeddings is used for generating embeddings for documents
model = ChatOpenAI(model="gpt-4", api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

In [20]:
# Initialize the retriever
def initialize_vectorstore(directory):
    """
    Initializes a vector store from the documents found in the specified directory.
    
    This function performs the following steps:
    1. Loads text documents from the given directory using a DirectoryLoader.
    2. Creates embeddings for the loaded documents using a predefined embedding model.
    3. Initializes a Chroma vector store with these embeddings.
    
    Parameters:
        directory (str): The path to the directory containing text files to be processed.
        
    Returns:
        vectorstore (Chroma): A Chroma vector store object containing the embeddings of the documents.
        docs (List[Document]): A list of Document objects loaded from the specified directory.
        
    Example:
        >>> vectorstore, docs = initialize_vectorstore("path/to/your/directory")
    """
    
    # Load documents from the specified directory using DirectoryLoader
    loader = DirectoryLoader(directory, glob="**/*.txt", loader_cls=TextLoader)
    docs = loader.load()  # Load all text documents matching the pattern
    
    # Create a Chroma vector store from the loaded documents and embeddings
    vectorstore = Chroma.from_documents(documents=docs, embedding=embedding)
    
    return vectorstore, docs

# Initialize the vector store and document list
vectorstore, docs = initialize_vectorstore("rag_data/website/organized_data")

# Set up the retriever using the vector store
retriever = vectorstore.as_retriever()

In [42]:
# Define the template for generating an answer based on context and a question
telekom_template = """You are an assistant for question-answering tasks for telekom.de help, providing answers to Telekom customers or potential customers. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer or if the provided documents do not contain relevant information, simply say that unfortunately, you cannot assist with this question and please visit telekom.de/hilfe for further assistance. 
Use up to four sentences and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

prompt_telekom = ChatPromptTemplate.from_template(telekom_template)

In [12]:
# Function to calculate cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    """
    Computes the cosine similarity between two vectors.
    
    Parameters:
    - vec1 (np.ndarray): The first vector.
    - vec2 (np.ndarray): The second vector.
    
    Returns:
    - float: The cosine similarity between vec1 and vec2.
    """
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2) if (norm_vec1 and norm_vec2) else 0.0

In [17]:
# Asynchronous function to print generated queries
async def print_generated_queries(question):
    """
    Generates and prints multiple search queries related to the input question.
    
    Parameters:
    - question (str): The input query for which related search queries are generated.
    """
    queries = generate_queries.invoke({"question": question})
    print("\nGenerated Questions:")
    for q in queries:
        print(f"{q}")

## Query Translation

## Multi-query

In [18]:
# Template for Generating Alternative Questions
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""

# Create a prompt template for generating multiple perspectives of the user's question
prompt_perspectives = ChatPromptTemplate.from_template(template)

# Define a pipeline for generating alternative queries
generate_queries = (
    prompt_perspectives 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))  # Split the generated output into individual queries
)

def get_unique_union(documents):
    """
    Returns a unique union of retrieved documents.

    This function takes a list of lists of documents, flattens it, and removes duplicates
    to ensure each document is unique.

    Args:
        documents (list of lists): A list where each element is a list of documents.

    Returns:
        list: A list of unique documents.
    """
    # Flatten the list of lists of documents
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Remove duplicates by converting to a set and then back to a list
    unique_docs = list(set(flattened_docs))
    # Deserialize the documents back into their original form
    return [loads(doc) for doc in unique_docs]

# Define the retrieval chain, which includes generating queries, retrieving documents, and removing duplicates
retrieval_chain = generate_queries | retriever.map() | get_unique_union

# Retrieve multiple documents based on the input question
multi_query_docs = retrieval_chain.invoke({"question": question})


def format_docs(docs, query_embedding):
    """
    Formats the retrieved documents with their source and cosine similarity score.

    This function takes a list of documents and formats them to include the source of each document
    and its cosine similarity to the query embedding.

    Args:
        docs (list): A list of documents retrieved from the database.
        query_embedding (numpy array): The embedding of the user's query.

    Returns:
        str: A formatted string containing the source, similarity score, and content of each document.
    """
    # Initialize a set to track unique sources
    unique_sources = set()
    formatted_docs = []

    for doc in docs:
        # Retrieve the source of the document from its metadata
        source = doc.metadata.get("source")
        # Check if the source is unique
        if source and source not in unique_sources:
            unique_sources.add(source)
            # Compute the embedding of the document's content
            document_embedding = embedding.embed_query(doc.page_content)
            # Calculate cosine similarity between the query and document embeddings
            similarity = cosine_similarity(query_embedding, document_embedding)
            # Use a placeholder message if the document content is empty
            content = doc.page_content.strip() or "This document content is empty."
            # Format the document's source, similarity score, and content
            formatted_docs.append(
                f"Source document: {source}\n\nCosine Similarity: {similarity:.4f}\n\n{content}"
            )

    # Join the formatted documents into a single string
    return "\n\n".join(formatted_docs)

# Define a retrieval and generation (RAG) chain for processing the question and context
rag_chain = (
    {"context": retrieval_chain, "question": itemgetter("question")} 
    | prompt_telekom
    | model
    | StrOutputParser()
)

async def retrieve_and_format_docs(question):
    """
    Asynchronously retrieves and formats documents for the given question.

    This function retrieves documents relevant to the user's question and formats them with their
    source information and cosine similarity scores.

    Args:
        question (str): The user's question.

    Returns:
        tuple: A tuple containing the answer and formatted documents.
    """
    # Compute the embedding for the user's question
    query_embedding = embedding.embed_query(question)
    # Format the retrieved documents with their cosine similarity scores
    formatted_docs = format_docs(multi_query_docs, query_embedding)
    
    try:
        # Attempt to retrieve an answer using the RAG chain asynchronously
        answer = await rag_chain.invoke({"context": formatted_docs, "question": question})
    except TypeError:
        # Fallback in case of TypeError, invoke the RAG chain synchronously
        answer = rag_chain.invoke({"context": formatted_docs, "question": question})
    
    # Return the answer and the formatted documents
    return answer, formatted_docs

async def main():
    """
    The main asynchronous function to run the complete flow.

    This function handles the process of generating alternative queries, retrieving and formatting
    documents, and printing the final answer along with the source documents.
    """
   
    # Retrieve and format documents, then get the answer
    answer, source_docs = await retrieve_and_format_docs(question)
    # Print the final answer
    print("Answer:", answer)
     # Generate and print alternative queries
    await print_generated_queries(question)
    # Print the source documents used for the answer
    print("\nSources:")
    print(source_docs)

# Execute the main function
await main()

Answer: You can cancel your contract through the customer center, the MeinMagenta App, a contact form, or by letter. If you choose to send a letter, include your name, address, the relevant phone number, and the customer number of the fixed network contract or the customer account of the mobile contract, and send it to: Telekom Deutschland GmbH, Kundenservice, 53171 Bonn. Alternatively, you can use the customer center or MeinMagenta App to select the contract you want to cancel and follow the prompts for cancellation.

Generated Questions:
1. How can I terminate my current contract?
2. What steps do I need to take to end my contract?
3. What is the process for canceling a contract?
4. How do I go about canceling my existing contract?
5. What are the necessary actions to cancel a contract?

Sources:
Source document: rag_data/website/organized_data/Vertrag & Rechnung/https_www_telekom_de_hilfe_vertrag_rechnung_vertrag_kuendigung_umzug_ausland.txt

Cosine Similarity: 0.7957

Source URL: h

## RAG-Fusion

In [24]:
# Define the template for generating multiple search queries based on a single input query.
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

# Create a chain for generating four related search queries
generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

# Function for Reciprocal Rank Fusion (RRF)
def reciprocal_rank_fusion(results: list[list], k=60):
    """
    Applies Reciprocal Rank Fusion (RRF) to combine multiple lists of ranked documents.
    
    Parameters:
    - results (list[list]): A list of lists where each inner list contains ranked documents.
    - k (int): An optional parameter for the RRF formula, default is 60.
    
    Returns:
    - list: A list of tuples where each tuple contains a document and its fused score.
    """
    
    # Initialize a dictionary to store the fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Serialize the document to a string format to use as a key
            doc_str = dumps(doc)
            # Initialize the document's score if not already present
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Update the document's score using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort documents based on their fused scores in descending order
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples
    return reranked_results

# Create a retrieval chain that generates queries, retrieves documents, and applies RRF
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
fusion_docs = retrieval_chain_rag_fusion.invoke({"question": question})

# Function to get embeddings for a document's content
async def get_document_embeddings(doc):
    """
    Retrieves the embeddings for a document's content asynchronously.
    
    Parameters:
    - doc (Document): The document object whose content embeddings are to be retrieved.
    
    Returns:
    - np.ndarray: The embeddings of the document's content.
    """
    return embedding.embed_query(doc.page_content)

# Function to format fusion_docs as a readable string with similarity scores
async def format_fusion_docs_with_similarity(fusion_docs):
    """
    Formats the fusion documents with their scores and cosine similarity to the question.
    
    Parameters:
    - fusion_docs (list[tuple]): A list of tuples containing documents and their scores.
    
    Returns:
    - str: A formatted string containing each document's source, fusion score, cosine similarity, and content.
    """
    formatted_docs = []
    question_embedding = embedding.embed_query(question)
    
    for doc, score in fusion_docs:
        doc_embedding = await get_document_embeddings(doc)
        similarity = cosine_similarity(question_embedding, doc_embedding)
        source = doc.metadata.get("source", "No source")
        content = doc.page_content
        formatted_docs.append(f"Source: {source}\nFusion Score: {score:.4f}\nCosine Similarity: {similarity:.4f}\nContent: {content}\n")
    
    return "\n".join(formatted_docs)


# Create a chain that uses context and question to generate an answer
rag_chain = (
    {"context": retrieval_chain_rag_fusion, "question": itemgetter("question")} 
    | prompt_telekom
    | model
    | StrOutputParser()
)

# Asynchronous function to retrieve and format documents, then get an answer
async def retrieve_and_format_docs(question):
    """
    Retrieves and formats documents, then obtains an answer to the question.
    
    Parameters:
    - question (str): The query for which answers and document formats are required.
    
    Returns:
    - tuple: A tuple containing the answer and the formatted documents.
    """
    formatted_docs = await format_fusion_docs_with_similarity(fusion_docs)
    
    try:
        # Attempt to get the answer asynchronously
        answer = await rag_chain.invoke({"context": formatted_docs, "question": question})
    except TypeError:
        # Fallback to synchronous invocation if asynchronous fails
        answer = rag_chain.invoke({"context": formatted_docs, "question": question})
    
    return answer, formatted_docs


# Main function to run the sequence of operations
async def main():
    """
    Main function to execute the entire process: generating queries, retrieving and formatting documents, and getting answers.
    """
    
    answer, formatted_docs = await retrieve_and_format_docs(question)
    print("Answer:", answer)
    await print_generated_queries(question)
    print("\nSources:")
    print(formatted_docs)  # Print the formatted version of fusion_docs with similarity scores

# Execute the main function
await main()

Answer: To get an eSIM, you can switch from your "classic" SIM card to an eSIM in a few minutes. Just log into your customer center and follow these steps: Go to your contract or contract details > My SIMs > Settings > Security check > Swap SIM > Change SIM card type > Continue > Select SIM > Follow the further instructions. Please note that a security check via an SMS code (mTAN) is required. The SMS can only be sent to a mobile phone number that belongs to your contract account.

Generated Questions:
1. How to activate eSIM on my phone?
2. Which mobile carriers support eSIM technology?
3. Benefits of using eSIM over traditional SIM cards
4. Steps to switch from physical SIM card to eSIM on my device

Sources:
Source: rag_data/website/organized_data/Mobilfunk/https_www_telekom_de_hilfe_mobilfunk_esim_geraetewechsel.txt
Fusion Score: 0.0651
Cosine Similarity: 0.8057
Content: Source URL: https://www.telekom.de/hilfe/mobilfunk/esim/geraetewechsel
Telekom > Hilfe & Service > Mobilfunk > e

## !!Decomposition
## Calismadi olmadi maalesef, asnwer sadece 3. sorunun cevabini veriyor, stratch den baska kaynakalara bakip cözüm bulmak lazim.

In [36]:
# Define prompts and chains
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answered in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

# Chain
generate_queries_decomposition = ( prompt_decomposition | model | StrOutputParser() | (lambda x: x.split("\n")))

# Run
questions = generate_queries_decomposition.invoke({"question":question})

In [37]:
questions

['1. "What is an eSIM and how does it work?"',
 '2. "How to get an eSIM?"',
 '3. "What devices support eSIM technology?"']

In [44]:
# Answer recursion
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""
decomposition_prompt = ChatPromptTemplate.from_template(template)

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()


q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | model
    | StrOutputParser())

    answer_decomposition = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer_decomposition)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [45]:
answer_decomposition

'Unfortunately, the provided documents do not contain specific information on what devices support eSIM technology. Please visit telekom.de/hilfe for further assistance.'

## Step Back
### cosine similarity eksik sadece calisiyor suan.

In [3]:
# Few Shot Examples
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]

# Transform examples into example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        few_shot_prompt,
        ("user", "{question}"),
    ]
)

# Generate step-back queries
generate_queries_step_back = prompt | model | StrOutputParser()
step_back_question = generate_queries_step_back.invoke({"question": question})

print(f"Original Question: {question}")
print(f"Step-Back Question: {step_back_question}")

# Response prompt template
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# Normal Context:
{normal_context}

# Step-Back Context:
{step_back_context}

# Original Question: {question}

# Answer:
"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

def get_retrieved_content(retrieved_documents):
    """Format retrieved documents as a string with source information."""
    seen_sources = set()
    content_list = []
    for doc in retrieved_documents:
        source = doc.metadata.get('source', 'Unknown')
        if source not in seen_sources:
            seen_sources.add(source)
            content = (
                f"Source: {source}\n"
                f"Content:\n{doc.page_content}\n"
                "------------------------------\n"
            )
            content_list.append(content)
    return "\n".join(content_list)

def format_retrieved_context(query):
    """Retrieve and format context for the given query."""
    # Retrieve documents using the 'invoke' method
    retrieved_docs = retriever.invoke(query)
    return get_retrieved_content(retrieved_docs)

# Construct the chain to retrieve and generate the response
chain = (
    {
        "normal_context": lambda x: format_retrieved_context(x["question"]),
        "step_back_context": lambda x: format_retrieved_context(x["step_back_question"]),
        "question": lambda x: x["question"],
    }
    | response_prompt
    | model
    | StrOutputParser()
)

# Execute the chain
result = chain.invoke({"question": question, "step_back_question": step_back_question})

# Display the final response
print("\nNormal Context:\n", format_retrieved_context(question))
print("\nStep-Back Context:\n", format_retrieved_context(step_back_question))
print("\nFinal Answer:\n", result)

Original Question: Glasfaser baglantisina sahibim, bilmeme gereken en önemli noktalar nelerdir?
Step-Back Question: Glasfaser bağlantısı hakkında genel bilgiye sahip olmam gerekiyor mu?

Normal Context:
 Source: rag_data/website/organized_data/Others/https_www_telekom_de_netz_glasfaser_neubauprojekte.txt
Content:
Source URL: https://www.telekom.de/netz/glasfaser/neubauprojekte

Question: Warum Telekom Glasfaser für Ihr privates Eigentum?
Answer: Mit einem Telekom Glasfaser-Anschluss sind Sie nicht an uns gebunden und können auch Produkte von anderen Anbietern nutzen.
Ein Glasfaser-Anschluss ist der neue Standard für die digitale Versorgung und steigert schon heute den Wert Ihrer Immobilie.
Profitieren Sie von der Erfahrung und Zuverlässigkeit der Telekom als Partner. Wir stehen Ihnen jederzeit zur Seite und sorgen für einen reibungslosen Ablauf.

Question: Worüber möchten Sie sich informieren?
Answer: Wer baut, muss rechtzeitig planen. In allen Fragen zum modernen Hausanschluss berät u

In [50]:
# Few Shot Examples
# This list provides example pairs of input questions and their corresponding step-back questions for model training.
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]

# Create a prompt template for examples.
# This template formats example messages for the model to learn from.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),  # Input from the user
        ("ai", "{output}"),    # Model's response to the input
    ]
)

# Create a few-shot prompt template that includes example prompts.
# This helps the model understand the context by providing example inputs and outputs.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# Define the final prompt template.
# This includes system instructions and integrates the few-shot prompt.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        few_shot_prompt,
        ("user", "{question}"),  # Input question from the user
    ]
)

# Generate step-back queries using the defined prompt.
# This involves processing the original question to generate a more general query.
generate_queries_step_back = prompt | model | StrOutputParser()
step_back_question = generate_queries_step_back.invoke({"question": question})


# Response prompt template
# This template is used to generate the final response based on the retrieved context and the original question.
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# Normal Context:
{normal_context}

# Step-Back Context:
{step_back_context}

# Original Question: {question}

# Answer:
"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

def get_retrieved_content(retrieved_documents):
    """
    Format retrieved documents as a string with source information.
    
    Args:
        retrieved_documents (list): List of documents retrieved based on the query.
        
    Returns:
        str: Formatted string containing source and content of retrieved documents.
    """
    seen_sources = set()  # Track unique sources
    content_list = []      # List to accumulate formatted content
    for doc in retrieved_documents:
        source = doc.metadata.get('source', 'Unknown')  # Get source of the document
        if source not in seen_sources:
            seen_sources.add(source)
            content = (
                f"Source: {source}\n"
                f"Content:\n{doc.page_content}\n"
                "------------------------------\n"
            )
            content_list.append(content)
    return "\n".join(content_list)

def format_retrieved_context(query):
    """
    Retrieve and format context for the given query.
    
    Args:
        query (str): The query for which context needs to be retrieved.
        
    Returns:
        str: Formatted string containing context relevant to the query.
    """
    # Retrieve documents using the 'invoke' method
    retrieved_docs = retriever.invoke(query)
    return get_retrieved_content(retrieved_docs)

# Construct the chain to retrieve and generate the response.
# This chain combines context retrieval and response generation.
chain = (
    {
        "normal_context": lambda x: format_retrieved_context(x["question"]),
        "step_back_context": lambda x: format_retrieved_context(x["step_back_question"]),
        "question": lambda x: x["question"],
    }
    | response_prompt
    | model
    | StrOutputParser()
)

# Execute the chain to get the final response.
result = chain.invoke({"question": question, "step_back_question": step_back_question})

# Display the final response along with normal and step-back contexts.
print("Answer:", result)
print(f"\n\nOriginal Question: {question}")
print(f"\nStep-Back Question: {step_back_question}")
print("\nNormal Context:\n", format_retrieved_context(question))
print("\nStep-Back Context:\n", format_retrieved_context(step_back_question))


Answer: To get an eSIM, you should follow a similar process to ordering a MultiSIM from Telekom, provided they offer eSIM services. Here is a general process:

1. Log into your customer center (Kundencenter) on the Telekom website.
2. Select the contract for which you want to order the eSIM by clicking on "Zum Vertrag" or "Vertragsdetails".
3. Scroll down to the "My SIMs" or "Meine SIMs" section.
4. Click on the button to order an eSIM (similar to the "MultiSIM bestellen" button for MultiSIM).
5. Follow the prompts for the "Sicherheitsüberprüfung" or security verification.
6. Finally, complete your order by clicking on a button similar to "Zahlungspflichtig bestellen".

Please note that the exact process and availability may vary, and you should consult the specific instructions provided by Telekom or your mobile service provider.


Original Question: I would like to get esim. What should I do?

Step-Back Question: What are the steps to acquire esim?

Normal Context:
 Source: rag_data/

## HyDE

In [11]:
# HyDE document generation
template = """You are creating professional and customer-focused web page content and texts for a major telecommunications provider like Telekom.de. 
Your content is very brief, very clear, and informative. Please write a text for the following question
Question: {question}
text:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

generate_docs_for_retrieval = (
    prompt_hyde | ChatOpenAI(temperature=0) | StrOutputParser() 
)

# Run HyDE generation
try:
    hyde_output = generate_docs_for_retrieval.invoke({"question": question})
    print(f"HyDE hypothetical answer:\n{hyde_output.strip()}\n")
except Exception as e:
    logger.error(f"Error generating documents for retrieval: {e}")
    raise

# Retrieve documents
try:
    retrieval_chain = generate_docs_for_retrieval | retriever 
    retrieved_docs = retrieval_chain.invoke({"question": question})
    
    # Print retrieved documents, deduplicated
    seen_sources = set()
    print("Retrieved sources:")
    for doc in retrieved_docs:
        source = doc.metadata.get('source', 'Unknown Source')
        if source not in seen_sources:
            seen_sources.add(source)
            print(f"\nDocument Source: {source}")
            print(f"Document Content:\n{doc.page_content.strip()}")
except Exception as e:
    logger.error(f"Error retrieving documents: {e}")
    raise

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | model
    | StrOutputParser()
)

try:
    final_answer = final_rag_chain.invoke({"context": retrieved_docs, "question": question})
    print(f"\nFinal RAG Answer:\n{final_answer.strip()}")
except Exception as e:
    logger.error(f"Error generating final RAG answer: {e}")
    raise

HyDE hypothetical answer:
Glasfaser bağlantısına sahip olmanın en önemli noktaları şunlardır: 
1. Yüksek hız ve güvenilirlik: Glasfaser bağlantısı, yüksek hızda veri iletimi sağlar ve kesintisiz bir internet deneyimi sunar.
2. Yüksek bant genişliği: Glasfaser bağlantısı, aynı anda birden fazla cihazın sorunsuz bir şekilde internete bağlanmasını sağlar.
3. Düşük gecikme süresi: Glasfaser bağlantısı, oyun oynama, video konferans yapma gibi uygulamalarda düşük gecikme süresi sunar.
4. Profesyonel kurulum ve destek: Glasfaser bağlantısının kurulumu ve sorun giderilmesi konusunda profesyonel destek alabilirsiniz.

Retrieved sources:

Document Source: rag_data/website/organized_data/Others/https_www_telekom_de_netz_glasfaser_vorteile.txt
Document Content:
Source URL: https://www.telekom.de/netz/glasfaser-vorteile

Question: Ist ein Glasfaser-Anschluss im Vergleich zu DSL sinnvoll?
Answer: Ja, einGlasfaser-Anschlussist aus mehreren Gründen sinnvoll:
• Im Gegensatz zu DSL ist Glasfaser alsInte

In [52]:
# HyDE Document Generation
# This section is responsible for creating professional and customer-focused content
# for a major telecommunications provider based on a given question.

# Define a template for generating content.
# The template specifies that the content should be brief, clear, and informative.
template = """You are creating professional and customer-focused web page content and texts for a major telecommunications provider like Telekom.de. 
Your content is very brief, very clear, and informative. Please write a text for the following question:
Question: {question}
text:"""

# Create a prompt template using the defined template.
# This template will be used to generate content for a given question.
prompt_hyde = ChatPromptTemplate.from_template(template)

# Define a chain to generate documents for retrieval.
# This chain uses the prompt template, a language model, and an output parser.
generate_docs_for_retrieval = (
    prompt_hyde | ChatOpenAI(temperature=0) | StrOutputParser()
)

# Run HyDE document generation to produce content for the given question.
# The try-except block handles potential errors during document generation.
try:
    hyde_output = generate_docs_for_retrieval.invoke({"question": question})
    print(f"HyDE hypothetical context:\n{hyde_output.strip()}\n")
except Exception as e:
    logger.error(f"Error generating documents for retrieval: {e}")
    raise

# Retrieve Documents
# This section retrieves documents based on the generated content and prints them.

# Define a chain to retrieve documents using the generated content.
# The chain combines the document generation process with a retriever.
try:
    retrieval_chain = generate_docs_for_retrieval | retriever 
    retrieved_docs = retrieval_chain.invoke({"question": question})
    
    # Print retrieved documents and deduplicate them based on source information.
    seen_sources = set()
    print("Retrieved sources:")
    for doc in retrieved_docs:
        source = doc.metadata.get('source', 'Unknown Source')  # Get the source of the document
        if source not in seen_sources:
            seen_sources.add(source)
            print(f"\nSource file: {source}")
            print(f"Document Content:\n{doc.page_content.strip()}")
except Exception as e:
    logger.error(f"Error retrieving documents: {e}")
    raise

# Define a chain to generate the final answer using the RAG process.
# The chain combines the prompt template, a language model, and an output parser.
final_rag_chain = (
    prompt_telekom
    | model
    | StrOutputParser()
)

# Generate the final answer using the RAG process.
# The try-except block handles potential errors during the final answer generation.
try:
    final_answer = final_rag_chain.invoke({"context": retrieved_docs, "question": question})
    print(f"\nFinal Answer:\n{final_answer.strip()}")
except Exception as e:
    logger.error(f"Error generating final RAG answer: {e}")
    raise

HyDE hypothetical answer:To get an eSIM with Telekom.de, simply visit our website or contact our customer service team. We will guide you through the process of activating your eSIM on your device. Enjoy the convenience of having a digital SIM card with Telekom.de.

Retrieved sources:

Source file: rag_data/website/organized_data/Mobilfunk/https_www_telekom_de_hilfe_mobilfunk_esim_bestellen_einrichten_neuvertrag_smartphone.txt
Document Content:
Source URL: https://www.telekom.de/hilfe/mobilfunk/esim/bestellen-einrichten/neuvertrag-smartphone
Telekom > Hilfe & Service > Mobilfunk > eSIM > Bestellen & Einrichten > Mit > Neuvertrag

Question: Warum habe ich keine SIM-Karte oder eSIM-Aktivierungscode zu meinem neuen Smartphone erhalten?
Answer: Wenn Sie einen neuen MagentaMobil Vertrag mit Smartphone abgeschlossen haben, wird Ihr Gerät automatisch über "eSIM direct" mit Ihrem Mobilfunk-Profil verknüpft.
Sie erhalten deshalb weder SIM-Karte noch eSIM QR-Code. Sobald das neue Smartphone eing